## chunking, vectorDB를 활용해 RAG구조를 구성하고 커스터마이징된 대화형 AI를 구현해보기

In [3]:
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
# langchain, openai가 연된된 라이브러리에서 pre-trained Embedding 모델 import
from langchain_openai import OpenAIEmbeddings

## 1) gpt-3.5-turbo 모델에게 일반적인 질의

In [5]:
MY_API_KEY = getpass.getpass("OpenAI API Key : ")

OpenAI API Key :  ········


In [7]:
client = OpenAI(api_key = MY_API_KEY)

In [9]:
completion = client.chat.completions.create(model = "gpt-3.5-turbo",
                                          messages = [{'role':'user','content':'Who is Son Heung Min?'}],
                                          max_tokens = 150,
                                          temperature = 0)
print(completion.choices[0].message.content)

Son Heung Min is a professional footballer from South Korea who currently plays as a forward for Tottenham Hotspur in the English Premier League. He is considered one of the best Asian footballers in the world and has achieved great success both at the club and international level. Son has won numerous individual awards and has been a key player for Tottenham Hotspur since joining the club in 2015.


## 2) RAG구조와 vectorDB를 활용한 질문

### 1) 참조할 데이터 로드 및 청킹

In [11]:
# 한 번에 여러개의 PDF 로드
loaders = [PyPDFLoader("data/기술보증기금과 한국경제.pdf"),
          PyPDFLoader("data/기술보증기금 주요업무.pdf")]

In [13]:
# 청킹을 위한 splitter 객체 생성
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                             chunk_overlap = 50,
                                             separators = ["\n\n", "\n", " "])

In [15]:
my_chunks = [] # 청크들을 보관할 리스트

for loader in loaders:
    # 2개의 PDF 파일에서 추출된 document객체들을 하나의 리스트에 합쳐줌.
    # load_and_split함수를 사용하면 반환되는 형식이 리스트이므로 extend 함수를 사용
    my_chunks.extend(loader.load_and_split(pdf_splitter))
print("number of chunks: ", len(my_chunks))
print(my_chunks)

number of chunks:  13
[Document(metadata={'producer': 'Microsoft® Word Office 365용', 'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'moddate': '2019-06-27T22:46:56+09:00', 'source': 'data/기술보증기금과 한국경제.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는

## 2) 임베딩 설정

In [17]:
# 임베딩 모델 API를 사용하여 임베딩 객체 생성
# text-embedding-3-small 모델의 최대 입력 가능 토큰 수는 8191개, 벡터의 길이(차원)은 1536차원
my_embedding = OpenAIEmbeddings(model = "text-embedding-3-small",
                               api_key=MY_API_KEY)
my_embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000282BC64DC10>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000282BCE884D0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [18]:
# 임베딩 예시
my_list = ['오늘은 비가오는 우중충한 날씨네요 가실 때 조심하세요!', "두번째 문장은 여기까지"]

temp = my_embedding.embed_documents(my_list)
print('length of my_list:', len(temp))
print('length of first sentence: ', len(temp[0]))
print('length of second sentence: ', len(temp[1]))
print(temp)

length of my_list: 2
length of first sentence:  1536
length of second sentence:  1536
[[0.06361066550016403, 0.005031932611018419, -0.06584056466817856, 0.01776091195642948, -0.012587166391313076, -0.020127730444073677, 0.00867996085435152, -0.013839037157595158, -0.04131172597408295, -0.03869062289595604, -0.04858822375535965, 0.06216319277882576, 0.015198489651083946, -0.01394661981612444, 0.04272007942199707, 0.029223350808024406, -0.011990571394562721, 0.016636185348033905, -0.010327931493520737, 0.01255782600492239, -0.07139573991298676, -0.026719611138105392, -0.023159604519605637, 0.018562892451882362, -0.043189533054828644, -0.005873032845556736, 0.01686112955212593, 0.019267069175839424, 0.011374416761100292, -0.026934776455163956, 0.0353066585958004, -0.040607549250125885, 0.04964448884129524, -0.06306297332048416, 0.008440345525741577, -0.021457841619849205, 0.021809929981827736, -0.022064216434955597, -0.029966648668050766, -0.006733694113790989, -0.03575655072927475, -0.05

## 3) Chroma DB 생성

In [16]:
# langchain연동 chroma DB 설치
# !pip install langchain-chroma

In [21]:
from langchain_chroma import Chroma

In [ ]:
my_directory = "/VectorStores"

# from_documents: 청킹된 문서(document 객체)의 정보 및 임베딩, 경로 정보로 Chroma DB생성
vectordb = Chroma.from_documents(documents = my_chunks, # 벡터화 시킬 청크 목록
                                embedding = my_embedding, # 임베딩 객체
                                persist_directory = my_directory) # 벡터 DB 저장 경로

### Chroma 객체로 사용할 수 있는 메소드 종류
- _collection.count: 수집된 벡터들의 개수 확인
- _collection.get: 수집된 벡터들의 정보 확인
- delete: 특정 id를 가진 벡터 삭제
- add_documents:여러 문서를 벡터 DB에 추가
- add_texts: 여러 텍스트를 벡터 DB에 추가
- similarity_search: 유사도 계산을 통해 입력 질의에 대해 가장 유사한 문제 검색

In [ ]:
# (1) DB 정보 확인
# 수집된 청크들 id, documents(원본 문서), metadatas(문서와 관련된 정보)등을 확인
vectordb._collection.get()

### (2) 기존 DB에 문서 추가하기

In [25]:
add_loaders = [PyPDFLoader("data/기술보증기금 주요업무_기술평가.pdf"),
          PyPDFLoader("data/기술보증기금 주요업무_기술보증.pdf")]

add_chuncks = []

for loader in add_loaders:
    my_chunks.extend(loader.load_and_split(pdf_splitter))
print("number of chunks to add: ", len(add_chuncks))
print(my_chunks)

number of chunks to add:  0
[Document(metadata={'producer': 'Microsoft® Word Office 365용', 'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'moddate': '2019-06-27T22:46:56+09:00', 'source': 'data/기술보증기금과 한국경제.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 

In [1]:
# vectordb에 문서 추가(추가 후 추가된 각 벡터에 대한 id값 정보가 출력됨.)
vectordb.add_documents(add_chuncks)

NameError: name 'vectordb' is not defined

In [3]:
vectordb._collection.count()

NameError: name 'vectordb' is not defined

In [5]:
vectordb._collection.get()

NameError: name 'vectordb' is not defined

In [ ]:
vectordb.delete(ids = [])
vectordb._collection.get()

In [ ]:
# 모든 데이터 삭제
vectordb.delete_collection()

In [ ]:
# 저장된 값이 없으면 에러 발생
vectordb._collection.count()

### 한번 생성된 DB 불러와서 사용하기(주피터 노트북 or 코랩을 꺼다 켰을 경우)
- 같은 경로에 DB를 계속 생성하면 중복된 데이터가 반복되어 저장됨.
- 따라서 한번 DB를 생성하고 데이터를 저장해뒀다면 다음 접속에서는 기존 DB를 불러와서 사용하는 방식으로 진행해야함.

In [ ]:
my_directory = "/VectorStores"

# from_documents: 청킹된 문서(document 객체)의 정보 및 임베딩, 경로 정보로 Chroma DB생성
vectordb = Chroma(documents = my_chunks, # 벡터화 시킬 청크 목록
                                embedding = my_embedding, # 임베딩 객체
                                persist_directory = my_directory) # 벡터 DB 저장 경로

## (4) 유사도 검색
- 간단한 질문에 대해 유사한 문서 검색

## (5) MMR 유사도 검색
- 사용자 쿼리에 가장 관련성이 높은 문서를 선택하지만 중복된 정보를 줄이고 다양한 정보를 제공하는 것을 목표로 하는 검색 방법

In [ ]:
# fetch_k: vectorDB로부터 가져올 유사도가 높은 문서 수
# k: 응답의 다양성을 고려하여 fetch_k중에서 반환할 문서 수(fetch_k에서 중복된 문서를 줄이고 다양성을 포함한 k개를 반환)
# lambda_mult: 0~1사이 실수값으로 관련성과 다양성 사이의 균형을 제어(0에 가까울 수록 다양성 우서, 1에 가까울 수록 관련성 우선)

search_mmr = vectordb.max_marginal_relevance_search(question, fetch_k = 6, k = 3, lambda_mut = 0.5)
search_mmr

# 일반적인 유사도 검색은 2,5,4page가 나왔는데 mmr 검색은 2,5,6 page가 나옴.